In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd

# import xgboost as xgb
# from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import lightgbm as lgb
import warnings
warnings.simplefilter("ignore")



tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

tf_df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    tf_df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(tf_df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)

kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [2]:
import json

list_bayesOpt = glob.glob('../models/SVD/LightGBM_BayesOpt_dec17/logs_shuffle*.json')

opt_best_df = pd.DataFrame()
for jsonFile in list_bayesOpt:
    with open(jsonFile) as f:
        optList = []
        for jsonObj in f:
            optDict = json.loads(jsonObj)
            optList.append(optDict)
        
        opt_df = pd.DataFrame(optList)
        opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
        opt_best_df = pd.concat([opt_best_df,opt_df.sort_values('target',ascending=False).iloc[0:2]])

opt_best_df.sort_values('target',ascending=False)
# opt_df = pd.DataFrame(optList)
# opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
# opt_df.sort_values('target',ascending=False)

,target,datetime,bagging_fraction,bagging_freq,batch_size,feature_fraction,learning_rate,min_gain_to_split,nFile,num_iterations,num_leaves
22,0.736777,"{'datetime': '2022-12-18 19:36:41', 'elapsed':...",0.8,2.0,10.0,0.636723,0.236670,0.007262,10.0,910.886433,17.133926
9,0.736412,"{'datetime': '2022-12-18 16:47:30', 'elapsed':...",0.8,2.0,10.0,0.520638,0.254145,0.050850,10.0,812.161532,19.901281
15,0.713089,"{'datetime': '2022-12-18 23:08:28', 'elapsed':...",0.8,2.0,10.0,0.789203,0.348110,0.009647,20.0,929.587079,28.791544
17,0.712356,"{'datetime': '2022-12-18 23:37:48', 'elapsed':...",0.8,2.0,10.0,0.642113,0.287042,0.022142,20.0,821.095331,28.000907
4,0.702660,"{'datetime': '2022-12-19 08:16:25', 'elapsed':...",0.8,2.0,10.0,0.528785,0.294654,0.077588,40.0,601.936084,9.456182
14,0.698809,"{'datetime': '2022-12-19 04:20:45', 'elapsed':...",0.8,2.0,10.0,0.738778,0.171203,0.059463,30.0,967.534580,29.607742
0,0.698502,"{'datetime': '2022-12-19 07:29:09', 'elapsed':...",0.8,2.0,10.0,0.369438,0.111302,0.068936,40.0,453.198220,20.451309
20,0.697060,"{'datetime': '2022-12-19 06:02:37', 'elapsed':...",0.8,2.0,10.0,0.711134,0.369049,0.018609,30.0,794.346555,29.146164


In [3]:
opt_best_df.mean()

target                 0.711958
bagging_fraction       0.800000
bagging_freq           2.000000
batch_size            10.000000
feature_fraction       0.617101
learning_rate          0.259022
min_gain_to_split      0.039312
nFile                 25.000000
num_iterations       786.343227
num_leaves            22.811132
dtype: float64

In [4]:
opt_best_df.median()

target                 0.707508
bagging_fraction       0.800000
bagging_freq           2.000000
batch_size            10.000000
feature_fraction       0.639418
learning_rate          0.270593
min_gain_to_split      0.036496
nFile                 25.000000
num_iterations       816.628432
num_leaves            24.226108
dtype: float64

In [5]:
def get_sim(df_hist, df_lookup, sim_file_list, score_name_list):
    df_hist['ListenYes'] = (df_hist['skip_2'] == False)*1
    df_hist['ListenYes'].replace(0, -1, inplace = True)
    df_hist = df_hist.groupby(['session_id', 'clus']).agg({'ListenYes':['sum']})
    df_hist = df_hist.reset_index()
    df_hist.columns = df_hist.columns.droplevel(level = 1) # take out the unwanted level
    df_pivot = pd.pivot_table(df_hist, values = 'ListenYes',index='session_id', columns='clus')
    df_pivot = df_pivot.fillna(0)
    
    
    for sim_file, score_name in zip(sim_file_list, score_name_list):
        sim_matrix = pd.read_csv(sim_file).drop(columns=['Unnamed: 0'])
        sim_matrix.columns = list(map(str, range(0,len(sim_matrix))))
        df_sim_session = df_pivot.dot(sim_matrix)/sim_matrix.sum()
        
        df_lookup[score_name] = df_sim_session.lookup(df_lookup['session_id'],df_lookup['clus'].astype(str))
    
    return df_lookup

In [6]:
# file_list = []
# for n in range(9):
#     file_list = file_list + glob.glob('../data/raw/training_set/log_'+str(n)+'*.csv')

# import random
# random.Random(23).shuffle(file_list) # randomly shuffle the list

In [7]:
file_list = pd.read_csv('../data/interim/train_valid_split_1217/train_file_list.csv', header = None)[0].tolist()
import random
random.Random(23).shuffle(file_list) # randomly shuffle the list
file_list

['../data/raw/training_set/log_8_20180830_000000000000.csv',
 '../data/raw/training_set/log_9_20180814_000000000000.csv',
 '../data/raw/training_set/log_3_20180913_000000000000.csv',
 '../data/raw/training_set/log_8_20180810_000000000000.csv',
 '../data/raw/training_set/log_8_20180720_000000000000.csv',
 '../data/raw/training_set/log_7_20180717_000000000000.csv',
 '../data/raw/training_set/log_2_20180826_000000000000.csv',
 '../data/raw/training_set/log_7_20180902_000000000000.csv',
 '../data/raw/training_set/log_3_20180908_000000000000.csv',
 '../data/raw/training_set/log_4_20180906_000000000000.csv',
 '../data/raw/training_set/log_4_20180815_000000000000.csv',
 '../data/raw/training_set/log_8_20180801_000000000000.csv',
 '../data/raw/training_set/log_6_20180907_000000000000.csv',
 '../data/raw/training_set/log_5_20180914_000000000000.csv',
 '../data/raw/training_set/log_7_20180715_000000000000.csv',
 '../data/raw/training_set/log_6_20180917_000000000000.csv',
 '../data/raw/training_s

In [8]:
def prep_dfs(file, tf_df):
    log_df = pd.read_csv(file)
    log_df = log_df.merge(kmean300_df)

    log_df_1 = log_df.loc[log_df['session_position']<=(log_df['session_length']/2)]
    log_df_1['hour_of_day'] = log_df_1['hour_of_day'].astype('float')
    log_df_1['premium'] = log_df_1['premium'].astype('bool')
    log_df_1['weekday'] = log_df_1['date'].astype('datetime64[ns]').dt.dayofweek
    log_df_1 = log_df_1.drop(columns = ['date'])
    log_df_1 = pd.get_dummies(log_df_1, columns=['hist_user_behavior_reason_end', 'hist_user_behavior_reason_start', 'context_type','weekday'], dtype = 'bool')
    log_df_1 = log_df_1.merge(tf_df.drop(columns = ['time_signature','mode','key']))
    
                     
    col_bool = log_df_1.columns[log_df_1.dtypes=='bool']
    col_nonbool = log_df_1.columns[log_df_1.dtypes!='bool'].drop(['session_id','track_id_clean','clus'])
    
    # the non-convertable values will be set to 0
    log_df_1[col_nonbool] = log_df_1[col_nonbool].apply(pd.to_numeric, errors='coerce', downcast = 'float').fillna(0).astype('float32')

    # aggregate the track history where ['skip_2']==True
    log_df_1_summary_skip2True = pd.concat([log_df_1.loc[log_df_1['skip_2']==True].groupby(['session_id'])[col_bool].agg(['mean']), 
                                            log_df_1.loc[log_df_1['skip_2']==True].groupby(['session_id'])[col_nonbool].agg(['mean', 'std', 'median'])],
                                            axis = 1)
    log_df_1_summary_skip2True.columns = log_df_1_summary_skip2True.columns.get_level_values(0)+'_sk2True_'+log_df_1_summary_skip2True.columns.get_level_values(1)
    
    # aggregate the track history where ['skip_2']==False
    log_df_1_summary_skip2False = pd.concat([log_df_1.loc[log_df_1['skip_2']==False].groupby(['session_id'])[col_bool].agg(['mean']), 
                                             log_df_1.loc[log_df_1['skip_2']==False].groupby(['session_id'])[col_nonbool].agg(['mean', 'std', 'median'])],
                                             axis = 1)
    log_df_1_summary_skip2False.columns = log_df_1_summary_skip2False.columns.get_level_values(0)+'_sk2False_'+log_df_1_summary_skip2False.columns.get_level_values(1)
    
    
    log_df_history = log_df_1[['session_id','track_id_clean','skip_2','clus']]


    half_cut = log_df['session_length']/2

    # need to at least include 2 trials, otherwise the log_df_1_summary will confound with all the tracks in the same session

    #1st trial in the 2nd half
    log_df_2_1 = log_df.loc[(log_df['session_position']>half_cut) & (log_df['session_position']<=half_cut+1)]
    log_df_2_1 = log_df_2_1[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2_1['weight'] = 1

    #2nd trial in the 2nd half
    log_df_2_2 = log_df.loc[(log_df['session_position']>half_cut+1) & (log_df['session_position']<=half_cut+2)]
    log_df_2_2 = log_df_2_2[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2_2['weight'] = 0.75

    log_df_2 = pd.concat([log_df_2_1,log_df_2_2])
    log_df_2 = log_df_2.merge(log_df_1_summary_skip2True, on='session_id')
    log_df_2 = log_df_2.merge(log_df_1_summary_skip2False, on='session_id')

    sim_file_list = ['../models/SVD/all_tracks/similarity/k300_CanbDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_CosSim.csv',
                     '../models/SVD/all_tracks/similarity/k300_LinCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_ManhDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_HammDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_SpearCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_KendCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_ChebDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_BrayDist.csv']
    score_name_list = ['CanbDist300', 'CosSim300','LinCorr300','ManhDist300','HammDist300','SpearCorr300','KendCorr300','ChebDist','BrayDist']

    return get_sim(log_df_history, log_df_2, sim_file_list, score_name_list)

In [9]:
import random
from timeit import default_timer as timer #to see how long the computation will take

nFile = 0
batch_size = 10
while nFile < len(file_list):
    start = timer()
    nFile += batch_size
    df_lookup_list = []
    for file in file_list[(nFile-batch_size):min(nFile, len(file_list))]:
        df_lookup_list.append(prep_dfs(file, tf_df))

    df_lookup = pd.concat(df_lookup_list)
    df_lookup = df_lookup.merge(tf_df.drop(columns = ['key','time_signature','mode']))
    
    # check whether the column names match with the previous training set
    if nFile>batch_size:
        prev_feature_names = lgb.Booster(model_file='../models/SVD/LightGBM_BayesOpt_dec17/LightGBM_incremental_training_dec19/boost'+str(int(nFile-batch_size))+'.txt').feature_name()
        if bool(set(prev_feature_names) - set(df_lookup.columns)): # if there are missing columns
            df_lookup[list(set(prev_feature_names) - set(df_lookup.columns))] = 0 # add the missed columns with 0

    dtrain = lgb.Dataset(df_lookup.drop(columns = ['session_id','track_id_clean','skip_2','weight']).astype('float32'), 
                     label=df_lookup['skip_2'],
                     weight = df_lookup['weight'],
                     free_raw_data=False) # https://lightgbm.readthedocs.io/en/latest/FAQ.html#error-messages-cannot-before-construct-dataset

    
    params = {'num_leaves': 17,
              'learning_rate':0.237,
              'metric': 'binary_error',
              'num_iterations':910,
              'bagging_fraction':0.8,
              'bagging_freq':2,
              'feature_fraction':0.637,
              'min_gain_to_split':0.007,
              'objective': 'binary',
              'force_row_wise': True,
              'num_threads': 5,
              'verbosity': 0,
              'tree_learner': 'voting'} #https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html
    
    if nFile == batch_size:
        bst = lgb.train(params, dtrain)
    else: # continue training on the previous model
        bst = lgb.train(params, dtrain, init_model='../models/SVD/LightGBM_BayesOpt_dec17/LightGBM_incremental_training_dec19/boost'+str(int(nFile-batch_size))+'.txt')
        
    bst.save_model('../models/SVD/LightGBM_BayesOpt_dec17/LightGBM_incremental_training_dec19/boost'+str(int(nFile))+'.txt')

    print('Runtime per batch: %0.2fs' % (timer() - start))

Runtime per batch: 745.19s
Runtime per batch: 789.75s
Runtime per batch: 939.17s
Runtime per batch: 877.30s
Runtime per batch: 1100.96s
Runtime per batch: 850.08s
Runtime per batch: 906.07s


KeyboardInterrupt: 